Notebooks này dùng để merge các file scraped_result_pX.csv đến file scraped_result_pY.csv trong data/raw/scraped/.

Chạy file này khi đang ở thư mục gốc là Lab1---Data-Visualization-

Các thông số cần cấu hình: 
- X
- Y

--> merge từ file X đến file Y

Ví dụ: X = 2; Y = 5 thì merge các file:

scraped_result_p2.csv,  scraped_result_p3.csv,  scraped_result_p4.csv,  scraped_result_p5.csv

--> output là: scraped_result_2to5.csv được lưu ở data/raw/merge/.

In [5]:
X = 2

Y = 50

from pathlib import Path
import pandas as pd
import re

cwd = Path.cwd().resolve()
project_root = None
for p in [cwd] + list(cwd.parents):
    if (p / "data" / "raw" / "scraped").exists():
        project_root = p
        break

if project_root is None:
    raise FileNotFoundError("Khong tim thay thu muc data/raw/scraped; hay mo notebook tu project root.")

raw_scraped_dir = project_root / "data" / "raw" / "scraped"
out_dir = project_root / "data" / "raw" / "merge"
out_dir.mkdir(parents=True, exist_ok=True)

if X > Y:
    raise ValueError("X phai nho hon hoac bang Y")

pattern = re.compile(r"scraped_results_p(\d+)\.csv$", re.IGNORECASE)
available = []
for path in raw_scraped_dir.glob("scraped_results_p*.csv"):
    m = pattern.search(path.name)
    if m:
        available.append((int(m.group(1)), path))

if not available:
    raise FileNotFoundError("Khong tim thay file nao trong data/raw/scraped/")

available.sort(key=lambda x: x[0])
available_pages = {p for p, _ in available}
selected = [(p, path) for p, path in available if X <= p <= Y]

if not selected:
    min_p, max_p = available[0][0], available[-1][0]
    raise FileNotFoundError(f"Khong co file trong khoang {X}-{Y}. Hien co: {min_p}-{max_p}")

frames = []
missing = []
for page in range(X, Y + 1):
    if page not in available_pages:
        missing.append(str(raw_scraped_dir / f"scraped_results_p{page}.csv"))

for page, path in selected:
    df = pd.read_csv(path)
    df["_source_file"] = path.name
    frames.append(df)

merged = pd.concat(frames, ignore_index=True)
out_path = out_dir / f"scraped_results_{X}to{Y}.csv"
merged.to_csv(out_path, index=False, encoding="utf-8-sig")

print(f"Da merge: {len(frames)} files -> {out_path}")
print(f"Tong so dong: {len(merged)}")
if missing:
    print("Cac file bi thieu:")
    for p in missing:
        print(p)

Da merge: 49 files -> D:\Vscode\Self\Data Science\Book Notes\Data Visualization\Lab\Lab1\Lab1---Data-Visualization-\data\raw\merge\scraped_results_2to50.csv
Tong so dong: 1460
